In [2]:
import pandas as pd

In [128]:
# Load Chicago crime datasets for 2023 and 2024
chicago_2023 = pd.read_csv('./datasets/Chicago_crime_incident_dataset_2023.csv')
chicago_2024 = pd.read_csv('./datasets/Chicago_crime_incident_dataset_2024.csv')

In [130]:
# Combine the datasets
chicago_combined = pd.concat([chicago_2023, chicago_2024], ignore_index=True)

In [10]:
chicago_combined.to_csv('./datasets/Chicago_crime_incident_dataset.csv', index=False)

In [28]:
# Create the 'Crime Description' column
chicago_combined['Crime Description'] = chicago_combined['Primary Type'] + ": " + chicago_combined['Description']

In [30]:
# Filter relevant columns and drop duplicates based on 'IUCR' (CRIME_CODE)
chicago_unique = chicago_combined[['IUCR', 'Crime Description']].drop_duplicates(subset='IUCR')

In [32]:
# Rename columns to match the SQL table schema
chicago_renamed = chicago_unique.rename(columns={
    'IUCR': 'CRIME_CODE',
    'Crime Description': 'CRIME_DESCRIPTION'
})

In [4]:
boston_data = pd.read_csv('./datasets/Boston_crime_incident_dataset.csv', low_memory=False)

In [43]:
boston_unique = boston_data[['OFFENSE_CODE', 'OFFENSE_DESCRIPTION']].drop_duplicates(subset='OFFENSE_CODE')

In [45]:
boston_renamed = boston_unique.rename(columns={
    'OFFENSE_CODE': 'CRIME_CODE',
    'OFFENSE_DESCRIPTION': 'CRIME_DESCRIPTION'
})

In [48]:
# Combine the two dataframes
combined_data = pd.concat([chicago_renamed, boston_renamed], ignore_index=True)

In [54]:
# Save the combined data to a new CSV file
combined_data.to_csv('./internal_datasets/crime_type_dataset.csv', index=False)

In [222]:
# Load datasets
boston_data = pd.read_csv('./datasets/Boston_crime_incident_dataset.csv', low_memory=False)
chicago_data = pd.read_csv('./datasets/Chicago_crime_incident_dataset.csv')

In [16]:
# Extract dates from Boston dataset
boston_data['date'] = pd.to_datetime(boston_data['OCCURRED_ON_DATE']).dt.date

In [18]:
# Extract dates from Chicago dataset (handling both Date and Updated On)
chicago_data['Date'] = pd.to_datetime(chicago_data['Date'], format='%m/%d/%Y %I:%M:%S %p').dt.date
chicago_data['Updated On'] = pd.to_datetime(chicago_data['Updated On'], format='%m/%d/%Y %I:%M:%S %p').dt.date

In [20]:
# Combine all date columns into one DataFrame
combined_dates = pd.concat([
    pd.DataFrame({'date': boston_data['date']}),
    pd.DataFrame({'date': chicago_data['Date']}),
    pd.DataFrame({'date': chicago_data['Updated On']})
])

In [22]:
# Remove duplicates and reset index
unique_dates = combined_dates.drop_duplicates().reset_index(drop=True)

In [24]:
# Extract year, month, and day into separate columns
unique_dates['year'] = pd.to_datetime(unique_dates['date']).dt.year
unique_dates['month'] = pd.to_datetime(unique_dates['date']).dt.month
unique_dates['day'] = pd.to_datetime(unique_dates['date']).dt.day

In [26]:
# Drop the original 'date' column if not needed
unique_dates = unique_dates[['year', 'month', 'day']]

In [28]:
# Save to a CSV file for upload to the database
unique_dates.to_csv('./internal_datasets/date_dimension.csv', index=False)

In [156]:
# Load Boston and Chicago datasets
boston_data = pd.read_csv('./datasets/Boston_crime_incident_dataset.csv', low_memory=False)
chicago_data = pd.read_csv('./datasets/Chicago_crime_incident_dataset.csv', low_memory=False)

In [135]:
# Boston dataset preparation
boston_location = boston_data[['DISTRICT', 'STREET']].copy()
boston_location['Ward'] = 0  # Default value for missing Ward
boston_location['Community_area'] = 0  # Default value for missing Community Area

In [138]:
# Rename columns to match Location table structure
boston_location.rename(columns={
    'DISTRICT': 'District',
    'STREET': 'Street'
}, inplace=True)

In [140]:
# Chicago dataset preparation
chicago_location = chicago_data[['District', 'Ward', 'Community Area', 'Block']].copy()
chicago_location.rename(columns={
    'Community Area': 'Community_area',
    'Block': 'Street'
}, inplace=True)

In [142]:
# Handle missing values and convert to appropriate data types
chicago_location['Ward'] = chicago_location['Ward'].fillna(0).astype(int)
chicago_location['Community_area'] = chicago_location['Community_area'].fillna(0).astype(int)
chicago_location['District'] = chicago_location['District'].fillna('')
chicago_location['Street'] = chicago_location['Street'].fillna('')

In [144]:
boston_location.fillna('', inplace=True)

In [146]:
# Combine datasets
combined_location = pd.concat([boston_location, chicago_location], ignore_index=True)

In [148]:
# Remove duplicates if needed
combined_location.drop_duplicates(inplace=True)

In [150]:
# Save to CSV
combined_location.to_csv('./internal_datasets/location_dataset.csv', index=False)

In [78]:
# Extract unique values from the REPORTING_AREA column
reporting_area = boston_data[['REPORTING_AREA']].drop_duplicates()

In [80]:
reporting_area['REPORTING_AREA'] = reporting_area['REPORTING_AREA'].fillna('')

In [82]:
# Rename the column to 'Code'
reporting_area.rename(columns={'REPORTING_AREA': 'Code'}, inplace=True)

In [84]:
# Save the result to a CSV file
reporting_area.to_csv('./internal_datasets/reporting_area.csv', index=False)

In [112]:
boston_day = boston_data[['OCCURRED_ON_DATE', 'YEAR', 'MONTH', 'DAY_OF_WEEK']].copy()

In [114]:
# Remove trailing and leading whitespaces in 'DAY_OF_WEEK'
boston_day['DAY_OF_WEEK'] = boston_day['DAY_OF_WEEK'].str.strip()

In [116]:
# Extract Day from OCCURRED_ON_DATE
boston_day['DAY'] = pd.to_datetime(boston_day['OCCURRED_ON_DATE']).dt.day

In [118]:
boston_day = boston_day[['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK']].drop_duplicates()

In [120]:
# Extract year, month, day, and day_of_week from Chicago data
chicago_data['Date'] = pd.to_datetime(chicago_data['Date'], format='%m/%d/%Y %I:%M:%S %p')
chicago_data['YEAR'] = chicago_data['Date'].dt.year
chicago_data['MONTH'] = chicago_data['Date'].dt.month
chicago_data['DAY'] = chicago_data['Date'].dt.day
chicago_data['DAY_OF_WEEK'] = chicago_data['Date'].dt.day_name()

In [122]:
chicago_day = chicago_data[['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK']].drop_duplicates()

In [124]:
# Combine Chicago and Boston data
combined_day = pd.concat([chicago_day, boston_day]).drop_duplicates()

In [126]:
# Save to CSV
combined_day.to_csv('./internal_datasets/day_dimension.csv', index=False)

In [225]:
# Convert 'Date' and 'Updated On' columns to datetime
chicago_data['Date'] = pd.to_datetime(chicago_data['Date'], errors='coerce')

In [229]:
chicago_data['Updated On'] = pd.to_datetime(chicago_data['Updated On'], errors='coerce')

In [231]:
# Extract Year, Month, Day, and Hour for 'Date'
chicago_data['Year'] = chicago_data['Date'].dt.year
chicago_data['Month'] = chicago_data['Date'].dt.month
chicago_data['Day'] = chicago_data['Date'].dt.day
chicago_data['Hour'] = chicago_data['Date'].dt.hour
chicago_data['Day_of_week'] = chicago_data['Date'].dt.day_name()

In [233]:
# Extract Year, Month, and Day for 'Updated On'
chicago_data['Updated_Year'] = chicago_data['Updated On'].dt.year
chicago_data['Updated_Month'] = chicago_data['Updated On'].dt.month
chicago_data['Updated_Day'] = chicago_data['Updated On'].dt.day

In [235]:
# Convert 'Arrest' and 'Domestic' to integers (1 for True, 0 for False)
chicago_data['Arrest'] = chicago_data['Arrest'].astype(int)
chicago_data['Domestic'] = chicago_data['Domestic'].astype(int)

In [237]:
# Handle missing values for Ward and Community Area
chicago_data.fillna({'Ward': 0, 'Community Area': 0}, inplace=True)

In [239]:
# Handle missing values and convert to appropriate data types
chicago_data['Ward'] = chicago_data['Ward'].fillna(0).astype(int)

In [241]:
chicago_data['Community Area'] = chicago_data['Community Area'].fillna(0).astype(int)

In [243]:
# Create the 'Crime Description' column
chicago_data['Crime Description'] = chicago_data['Primary Type'] + ": " + chicago_data['Description']

In [245]:
# Rename columns to match staging table conventions
chicago_data.rename(columns={
    'Case Number': 'Case_Number',
    'Primary Type': 'Primary_Type',
    'Crime Description': 'Crime_Description',
    'Location Description': 'Location_Description',
    'Community Area': 'Community_Area',
    'Updated On': 'Updated_On',
    'FBI Code': 'FBI_Code'
}, inplace=True)

In [251]:
# Select required columns
columns_to_keep = [
    'ID', 'Case_Number', 'Date', 'Year', 'Month', 'Day', 'Day_of_week', 'Hour', 'Block', 'IUCR', 
    'Primary_Type', 'Description', 'Crime_Description', 'Location_Description', 'Arrest', 'Domestic', 
    'Beat', 'District', 'Ward', 'Community_Area', 'FBI_Code', 'Updated_On', 
    'Updated_Year', 'Updated_Month', 'Updated_Day'
]

In [253]:
chicago_data = chicago_data[columns_to_keep]

In [255]:
# Save the processed data to a CSV file
chicago_data.to_csv('./internal_datasets/chicago_staging.csv', index=False)

In [6]:
# Remove trailing and leading whitespaces in 'DAY_OF_WEEK'
boston_data['DAY_OF_WEEK'] = boston_data['DAY_OF_WEEK'].str.strip()

In [8]:
# Convert 'OCCURED_ON_DATE' to datetime
boston_data['OCCURRED_ON_DATE'] = pd.to_datetime(boston_data['OCCURRED_ON_DATE'], errors='coerce')

In [10]:
# Extract DAY from 'OCCURED_ON_DATE'
boston_data['DAY'] = boston_data['OCCURRED_ON_DATE'].dt.day

In [12]:
# Select only the required columns
columns_to_keep = [
    '_id', 'INCIDENT_NUMBER', 'OFFENSE_CODE', 'OFFENSE_DESCRIPTION', 'DISTRICT', 
    'REPORTING_AREA', 'SHOOTING', 'OCCURRED_ON_DATE', 'YEAR', 'MONTH', 
    'DAY_OF_WEEK', 'HOUR', 'STREET', 'DAY'
]

In [14]:
boston_data = boston_data[columns_to_keep]

In [16]:
# Save the processed data to a CSV file
boston_data.to_csv('./internal_datasets/boston_staging.csv', index=False)